<a href="https://colab.research.google.com/github/Mithula-cbw/Car-Price-Prediction/blob/main/notebooks/01_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Car Price Prediction Using Machine Learning**

Group Assignment 02 - CCS3012 - Data Analytics

---

### **Group 11**
-  **FC211034 - N.D. Samararathne Kodikara**
-  **FC211013 - N.W.V. Tharindu Pabasara**
-  **FC211025 - W.M.M.C.B. Wijesundara**



---

### **Supervisor**
**Ms. Dilmi Praveena**  
*Faculty of Computing*  
*University of Sri Jayewardenepura*

---


# Exploratory Data Analysis for Car Price Prediction

In this notebook, we explore the structure and characteristics of the used car dataset to better understand the factors influencing vehicle pricing. This initial analysis includes inspecting data types, handling missing values, visualizing distributions, and identifying potential outliers and correlations. These insights will guide feature selection and model development in subsequent stages.


> 📌   **Objective summery**
- Understand the structure dataset.
- Explore the distribution of numerical and categorical features.
- Identify missing values, outliers, and data inconsistencies.
- Discover relationships between features and the target variable (Price).
- Generate insights to support effective preprocessing and cleaning.


In [1]:
#First we have to Import necessary libraries for data manipulation and visualization.

# Data Manipulation and Utilities
import pandas as pd     # For data manipulation and analysis.
import numpy as np      # For numerical operations.
import re               # For regular expressions.
import warnings         # For managing warnings.

# Data Visualization
import matplotlib.pyplot as plt     # For basic data visualization.
import seaborn as sns               # For statistical data visualization.

In [2]:
# Next we load the dataset.
df = pd.read_csv("./Data/raw/car_price_prediction.csv")

In [3]:
# Prints the first 5 rows of the DataFrame (Provides a quick look at the dataset's content).
df.head()

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4


💡 **Observations:**  
All the column names appear to be readable and meaningful.  
At first glance, the Doors column shows inconsistencies such as entries like "04-May," indicating data quality issues. The Levy column also contains missing values marked by dashes (-). Overall, the dataset features a mix of numerical and categorical variables that will require cleaning before analysis.


In [4]:
# Find the dimensions of the dataset
df.shape

(19237, 18)

📝 The dataset contains 19,237 rows and 18 columns.

In [5]:
from prettytable import PrettyTable # Python library to display tabular data in a nicely formatted ASCII table

# Prints the name of each column in the dataset, the number of non-null values it contains, and its data type.
def df_info(df):
    table = PrettyTable()
    table.field_names = ["Column", "Non-Null Count", "Dtype"]

    for col in df.columns:
        non_null_count = df[col].count()
        dtype = df[col].dtype
        table.add_row([col, non_null_count, dtype])

    print(table)

df_info(df)

+------------------+----------------+---------+
|      Column      | Non-Null Count |  Dtype  |
+------------------+----------------+---------+
|        ID        |     19237      |  int64  |
|      Price       |     19237      |  int64  |
|       Levy       |     19237      |  object |
|   Manufacturer   |     19237      |  object |
|      Model       |     19237      |  object |
|    Prod. year    |     19237      |  int64  |
|     Category     |     19237      |  object |
| Leather interior |     19237      |  object |
|    Fuel type     |     19237      |  object |
|  Engine volume   |     19237      |  object |
|     Mileage      |     19237      |  object |
|    Cylinders     |     19237      | float64 |
|  Gear box type   |     19237      |  object |
|   Drive wheels   |     19237      |  object |
|      Doors       |     19237      |  object |
|      Wheel       |     19237      |  object |
|      Color       |     19237      |  object |
|     Airbags      |     19237      |  i

💡 **Observations:**  
- All columns have 19,237 non-null values indicating no missing values. However, earlier checks revealed some missing or placeholder values that require further investigation and cleaning.
- Several columns expected to be numeric (like Levy, Mileage, and Engine volume) are currently of type object, which means they may contain non-numeric characters or inconsistent formatting.

In [6]:
# Prints basic statistics (mean, standard deviation, min, max, etc.)
df.describe(include=[np.number])      # For  numeric data types


,ID,Price,Prod. year,Cylinders,Airbags
count,1.923700e+04,1.923700e+04,19237.000000,19237.000000,19237.000000
mean,4.557654e+07,1.855593e+04,2010.912824,4.582991,6.582627
std,9.365914e+05,1.905813e+05,5.668673,1.199933,4.320168
min,2.074688e+07,1.000000e+00,1939.000000,1.000000,0.000000
25%,4.569837e+07,5.331000e+03,2009.000000,4.000000,4.000000
50%,4.577231e+07,1.317200e+04,2012.000000,4.000000,6.000000
75%,4.580204e+07,2.207500e+04,2015.000000,4.000000,12.000000
max,4.581665e+07,2.630750e+07,2020.000000,16.000000,16.000000


💡 **Observations:**  
- Price has a minimum value of $1 and a maximum that’s over 50 times the mean. Suggests outliers.
- Production year ranges from 1939 to 2020 — likely contains outliers or vintage cars.
- Cylinders has a Max value of 16, which could be rare or performance vehicles.  
- Some vehicles seems to have 0 airbags, which might indicate: Older models, Missing or misreported values.
- **ID** is a unique identifier for each row and does not carry predictive value.


In [7]:
# Let's see how many rows with these extreme values
# Production year:
production_year_40 = df[df['Prod. year'] < 1980].shape        # More than 40 yo cars

count_40 = production_year_40[0]
percentage_40 = (count_40 / df.shape[0]) * 100

print(f"Matches found: {count_40} ({percentage_40:.4f}%)")

Matches found: 23 (0.1196%)


> 💡 Only 23 rows (~0.11%) have a production year before 1980.  
> *🧠 These likely represent vintage or incorrectly entered data and can be safely removed to prevent skewing the analysis.*

In [8]:
# Cylinders:
cylinderes_16 = df[df['Cylinders'] == 16].shape

count_16 = cylinderes_16[0]
percentage_16 = (count_16 / df.shape[0]) * 100

print(f"Matches found: {count_16} ({percentage_16:.4f}%)")


Matches found: 5 (0.0260%)


>💡 Only 5 rows match — these likely represent performance vehicles.  
>*🧠 Since the model focuses on the general car market, they should be removed to avoid skewing the analysis.*

In [9]:

# Airbags:
airbags_0 = df[df['Airbags'] == 0].shape

count_0 = airbags_0[0]
percentage_0 = (count_0 / df.shape[0]) * 100

print(f"Matches found: {count_0} ({percentage_0:.4f}%)")

Matches found: 2405 (12.5019%)


> 💡 2,405  rows (~12.50%) have 0 airbags.  
> *🧠 This is a sizable portion of the data. Further inspection is needed before deciding how to handle them.*

In [10]:
# Prints basic statistics (mean, standard deviation, min, max, etc.)
df.describe(include='object')      # For  numeric data types

,Levy,Manufacturer,Model,Category,Leather interior,Fuel type,Engine volume,Mileage,Gear box type,Drive wheels,Doors,Wheel,Color
count,19237,19237,19237,19237,19237,19237,19237,19237,19237,19237,19237,19237,19237
unique,559,65,1590,11,2,7,107,7687,4,3,3,2,16
top,-,HYUNDAI,Prius,Sedan,Yes,Petrol,2,0 km,Automatic,Front,04-May,Left wheel,Black
freq,5819,3769,1083,8736,13954,10150,3916,721,13514,12874,18332,17753,5033


💡 **Observations:**  
- Most columns in the dataset are categorical.  

- The **Levy** column’s top value is '-', likely indicating no levy.

- **Mileage** has a top value of 0. can be new vehicles or placeholders. Need further analyses.

- The **Doors** column contains only 3 unique values but all of them has unusual values like "04-May".  
🧠 suggesting formatting errors that need cleaning.

- Several key columns show strong class imbalance in their major categories.

- Column **Model** has high cardinality (1051 unique), which may require special encoding techniques.

## Handling Duplicates
**Before diving deeper, we will start with some basic cleaning steps to improve data quality.**

In [11]:
duplicates_count = df.duplicated().sum()

def check_duplicates(df, label=""):
    duplicates_count = df.duplicated().sum()
    percentage = (duplicates_count / df.shape[0]) * 100
    if duplicates_count > 0:
        print(f"[{label}] Number of duplicate rows: {duplicates_count} ({percentage:.4f}%)")
    else:
        print(f"[{label}] No duplicate rows found.")


check_duplicates(df)


[] Number of duplicate rows: 313 (1.6271%)


In [12]:
# 📌Duplicates are not needed in the dataset. let's remove them first.
df_cleaned_1 = df.drop_duplicates(inplace=False)

check_duplicates(df_cleaned_1)

[] No duplicate rows found.


In [13]:
# Let's check if there are any duplicates after we remove the uniqe ID
# it checks if the dataset has logical duplicates (i.e., same car data under different IDs

# 📌Drop the unique ID column
df_no_id = df_cleaned_1.drop(columns='ID')

# Check for logical duplicates (same car data, different ID)
check_duplicates(df_no_id)


[] Number of duplicate rows: 3199 (16.9045%)


In [14]:
# 📌We can drop the ID column going forward, as it is a unique identifier with no value for modeling.

df_cleaned_1 = df_no_id.drop_duplicates()

check_duplicates(df_cleaned_1)


[] No duplicate rows found.


In [15]:
# Dataset shape after dealing with duplicates
df_cleaned_1.shape

(15725, 17)

## Handling Missing and Incorrect Data
**Placeholder values (like '-' in Levy) and inconsistent entries (e.g., in Doors) needs to be cleaned.**

### Handle **Levy** column first.

In [16]:
# Print missing data summery
df_cleaned_1.isnull().sum()

Price               0
Levy                0
Manufacturer        0
Model               0
Prod. year          0
Category            0
Leather interior    0
Fuel type           0
Engine volume       0
Mileage             0
Cylinders           0
Gear box type       0
Drive wheels        0
Doors               0
Wheel               0
Color               0
Airbags             0
dtype: int64

💡 **Observations:**  
- There are no missing values in the form of NaN.

- However, placeholder values like '-' were found in the Levy column, indicating missing or incomplete data that requires handling.

In [17]:
# Let's check how many missing values in levy column
print(f"Data type of the column: {df_cleaned_1['Levy'].dtype}\n")
print("Count of each unique values:\n")
df_cleaned_1['Levy'].value_counts()

Data type of the column: object

Count of each unique values:



Levy
-       5684
765      387
891      387
639      340
640      336
        ... 
2308       1
4860       1
1641       1
1045       1
1901       1
Name: count, Length: 559, dtype: int64

💡 **Observations:**  
- The **Levy** column appears to be numerical, but it's currently stored as an object type.

- Around 36.15% of the entries are placeholder values ('-'),
 >*🧠 likely indicating vehicles with no levy, so these can be safely replaced with 0.*


In [18]:
# 📌Let's replace the '-' with 0 and assing to a new dataframe.
# 📌And the datatype should be converted to float since Levy represents continuous numeric values.

df_cleaned_2 = df_cleaned_1.copy()

df_cleaned_2['Levy'] = df_cleaned_1['Levy'].replace('-', 0)
df_cleaned_2['Levy'] = df_cleaned_2['Levy'].astype(float)

print(f"Data type of the column: {df_cleaned_2['Levy'].dtype}\n")
print("Summery of Levy column after cleaning:\n")
df_cleaned_2['Levy'].describe()


Data type of the column: float64

Summery of Levy column after cleaning:



count    15725.000000
mean       561.555421
std        559.384534
min          0.000000
25%          0.000000
50%        639.000000
75%        862.000000
max      11714.000000
Name: Levy, dtype: float64

### Handle **Doors** column next

In [19]:
# Doors column had wrong data

print(f"Uniqe values in the column: {df_cleaned_2['Doors'].unique()}\n")
print("Count of each unique values:\n")
df_cleaned_2['Doors'].value_counts()

Uniqe values in the column: ['04-May' '02-Mar' '>5']

Count of each unique values:



Doors
04-May    14855
02-Mar      746
>5          124
Name: count, dtype: int64

💡 **Observations:**

- The Doors column contains inconsistent values like "04-May" and "02-Mar" which likely resulted from incorrect formatting or entry.

 > *🧠 The intended values appear to represent door counts such as 2-3, 4-5, and >5.  
 This column requires data cleaning to standardize the format and ensure correct interpretation.*

In [20]:
# 📌Let's replace the incorrect values in the 'Doors' column with correct, standardized categories

df_cleaned_3 = df_cleaned_2.copy()
df_cleaned_3['Doors'] = df_cleaned_2['Doors'].replace({'04-May': '4-5', '02-Mar': '2-3'})

# confirm the changes.
df_cleaned_3['Doors'].value_counts()

Doors
4-5    14855
2-3      746
>5       124
Name: count, dtype: int64

### Handle **Mileage** column next

In [21]:
# check the values
df_cleaned_3['Mileage'].value_counts()

Mileage
0 km         608
200000 km    181
150000 km    157
160000 km    119
180000 km    115
            ... 
104200 km      1
120455 km      1
147790 km      1
25038 km       1
133225 km      1
Name: count, Length: 7687, dtype: int64

💡 **Observations:**

- The **Mileage** column contains 7,687 unique values and is currently of type object, though it represents numeric data.

- 608 entries (3.86%) have a value of "0 km", which could indicate brand new cars or be placeholders for missing data.
>*🧠 This column should be converted to numeric for analysis, and "0" values may need further investigation or conditional handling.*

In [22]:
# 📌 Let's make Mileage numeric.
# First we have to remove the ' km' part.
df_cleaned_4 = df_cleaned_3.copy()
df_cleaned_4['Mileage'] = df_cleaned_4['Mileage'].str.replace(' km', '', regex=False)

# Then change datatype to numeric.
df_cleaned_4['Mileage'] = pd.to_numeric(df_cleaned_4['Mileage'], errors='coerce')

# Check if the changes have been made correctly.
print(f"New datatype of the column Mileage: {df_cleaned_4['Mileage'].dtype}")


New datatype of the column Mileage: int64
